# Neural Network SMS Text Classifier

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called predict_message that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the SMS Spam Collection dataset. The dataset has already been grouped into train data and test data.

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import sequence
tfds.disable_progress_bar()

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

--2024-02-15 04:43:02--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.006s  

2024-02-15 04:43:02 (53.6 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2024-02-15 04:43:02--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 104.26.2.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==============

In [ ]:
train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
# Load the train dataset
train_df = pd.read_csv(train_file_path, sep="\t", header=None, names=["type", "content"])
train_df.dropna(inplace=True)
train_df.head()

,type,content
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...


In [ ]:
# Load the test dataset
test_df = pd.read_csv(test_file_path, sep="\t", header=None, names=["type", "content"])
test_df.dropna(inplace=True)
test_df.head()

,type,content
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?


In [ ]:
# Map "ham" to 0 and "spam" to 1 in the "type" column of train and test DataFrames
train_df["type"] = train_df["type"].map({"ham": 0, "spam": 1})
test_df["type"] = test_df["type"].map({"ham": 0, "spam": 1})

print(train_df.head())
print(test_df.head())

   type                                            content
0     0  ahhhh...just woken up!had a bad dream about u ...
1     0                           you can never do nothing
2     0  now u sound like manky scouse boy steve,like! ...
3     0  mum say we wan to go then go... then she can s...
4     0  never y lei... i v lazy... got wat? dat day ü ...
   type                                            content
0     0  i am in hospital da. . i will return home in e...
1     0         not much, just some textin'. how bout you?
2     0  i probably won't eat at all today. i think i'm...
3     0  don‘t give a flying monkeys wot they think and...
4     0                                who are you seeing?


In [ ]:
# Extracting labels and messages from the training DataFrame
train_labels = train_df["type"].values
train_messages = train_df["content"].values

# Creating a TensorFlow dataset from the messages and labels
train_dataset = tf.data.Dataset.from_tensor_slices((train_messages, train_labels))

train_dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
# Extracting labels and messages from the test DataFrame
test_labels = test_df["type"].values
test_messages = test_df["content"].values

# Creating a TensorFlow dataset from the messages and labels
test_dataset = tf.data.Dataset.from_tensor_slices((test_messages, test_labels))

# Printing the element specification of the test dataset
print(test_dataset.element_spec)

(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))


In [ ]:
# Define constants for buffer size and batch size
BUFFER_SIZE = 100
BATCH_SIZE = 32

# Shuffle, batch, and prefetch the training dataset
train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Batch and prefetch the test dataset
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

In [ ]:
# Define TextVectorization parameters
vec = TextVectorization(
    output_mode='int',  # Output mode is set to 'int' to represent words as integers
    max_tokens=1000,     # Maximum number of tokens to keep in the vocabulary
    output_sequence_length=1000,  # Output sequence length after padding/truncation
)

# Adapt the TextVectorization layer to the training dataset
vec.adapt(train_dataset.map(lambda text, label: text))

In [ ]:
model = tf.keras.Sequential([
    vec,
    tf.keras.layers.Embedding(
        len(vec.get_vocabulary()),
        64,
        mask_zero=True,
    ),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.Dense(1)
])


model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-4),
    metrics=['accuracy'],
)
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization_5 (Text  (None, 1000)              0         
 Vectorization)                                                  
                                                                 
 embedding_5 (Embedding)     (None, 1000, 64)          64000     
                                                                 
 bidirectional_10 (Bidirect  (None, 1000, 128)         66048     
 ional)                                                          
                                                                 
 bidirectional_11 (Bidirect  (None, 64)                41216     
 ional)                                                          
                                                                 
 dense_10 (Dense)            (None, 64)                4160      
                                                      

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    validation_steps=30,
    epochs=10,
)

Epoch 1/10
131/131 [==============================] - 413s 3s/step - loss: 0.5936 - accuracy: 0.8660 - val_loss: 0.4769 - val_accuracy: 0.8604
Epoch 2/10
131/131 [==============================] - 388s 3s/step - loss: 0.3613 - accuracy: 0.8660 - val_loss: 0.2127 - val_accuracy: 0.8604
Epoch 3/10
131/131 [==============================] - 382s 3s/step - loss: 0.1423 - accuracy: 0.9407 - val_loss: 0.0987 - val_accuracy: 0.9698
Epoch 4/10
131/131 [==============================] - 381s 3s/step - loss: 0.0784 - accuracy: 0.9799 - val_loss: 0.0717 - val_accuracy: 0.9833
Epoch 5/10
131/131 [==============================] - 382s 3s/step - loss: 0.0595 - accuracy: 0.9847 - val_loss: 0.0640 - val_accuracy: 0.9865
Epoch 6/10
131/131 [==============================] - 381s 3s/step - loss: 0.0478 - accuracy: 0.9873 - val_loss: 0.0594 - val_accuracy: 0.9865
Epoch 7/10
131/131 [==============================] - 386s 3s/step - loss: 0.0384 - accuracy: 0.9895 - val_loss: 0.0577 - val_accuracy: 0.9885

In [ ]:
# Evaluate the model on the test dataset and get the loss and accuracy
test_loss, test_accuracy = model.evaluate(test_dataset)

print(f'Test Loss: {test_loss}')
print(f'Test Accuracy: {test_accuracy}')

44/44 [==============================] - 25s 580ms/step - loss: 0.0674 - accuracy: 0.9856
Test Loss: 0.06735878437757492
Test Accuracy: 0.9856321811676025


In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(input_text):
    # Predict the probability of input text being spam
    spam_probability = model.predict([input_text])
    print("Predicted spam probability:", spam_probability)

    # Extract the probability value
    probability_value = spam_probability[0][0]

    # Determine the label based on the probability
    label = "ham" if probability_value < 0.5 else "spam"

    # Return the probability and label
    return [probability_value, label]

pred_text = "how are you doing today?"
prediction = predict_message(pred_text)
print(prediction)

1/1 [==============================] - 0s 218ms/step
Predicted spam probability: [[-6.8771]]
[-6.8771, 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


1/1 [==============================] - 0s 214ms/step
Predicted spam probability: [[-6.8771]]
1/1 [==============================] - 0s 233ms/step
Predicted spam probability: [[1.7352095]]
1/1 [==============================] - 0s 221ms/step
Predicted spam probability: [[-13.004294]]
1/1 [==============================] - 0s 225ms/step
Predicted spam probability: [[4.8033986]]
1/1 [==============================] - 0s 214ms/step
Predicted spam probability: [[6.185276]]
1/1 [==============================] - 0s 213ms/step
Predicted spam probability: [[-11.464822]]
1/1 [==============================] - 0s 213ms/step
Predicted spam probability: [[-11.517773]]
You passed the challenge. Great job!
